In [15]:
import $ivy.`io.prometheus:simpleclient:0.8.1`
import $ivy.`io.prometheus:simpleclient_common:0.8.1`
import $ivy.`dev.zio::zio:1.0.0-RC18-2`

import $ivy.$                                 

import $ivy.$                                        

import $ivy.$                          

In [16]:
// This is a Java Library
import io.prometheus.client.CollectorRegistry
import io.prometheus.client.{ Counter, Histogram }
import io.prometheus.client.exporter.common.TextFormat

CollectorRegistry.defaultRegistry.clear()

// Spread operator => :_*
val c = Counter
    .build()
    .name("PrometheusCounter")
    .labelNames(Array("class", "method"): _*)
    .help(s"Sample prometheus counter")
    .register()

val h = Histogram
    .build()
    .name("PrometheusHistogram")
    .labelNames(Array("class", "method"): _*)
    .linearBuckets(0.0, 0.2, 5)
    .help(s"Sample prometheus histogram")
    .register()

import io.prometheus.client.CollectorRegistry

import io.prometheus.client.{ Counter, Histogram }

import io.prometheus.client.exporter.common.TextFormat


c: Counter = io.prometheus.client.Counter@79b3cb2a
h: Histogram = io.prometheus.client.Histogram@44edcab0

In [ ]:
c.labels(Array("Users", "getTotal"):_*).inc()
c.labels(Array("Users", "getUsers"):_*).inc(3)

h.labels(Array("Users", "getTotal"):_*).observe(0.32)
h.labels(Array("Users", "getUsers"):_*).observe(0.45)

In [17]:
import java.io.StringWriter

def write004(registry: CollectorRegistry) = {
    val writer = new StringWriter
    TextFormat.write004(writer, registry.metricFamilySamples)
    writer.toString
}

write004(CollectorRegistry.defaultRegistry)

import java.io.StringWriter


defined function write004
res16_2: String = """# HELP PrometheusCounter Sample prometheus counter
# TYPE PrometheusCounter counter
# HELP PrometheusHistogram Sample prometheus histogram
# TYPE PrometheusHistogram histogram
"""

In [18]:
import zio.console.putStrLn
import zio.console.Console
import zio.Runtime
import zio.Task

val writeT004: CollectorRegistry => Task[String] = registry => Task {
    val writer = new StringWriter
    TextFormat.write004(writer, registry.metricFamilySamples)
    writer.toString
}

val externalRegistry = Task.effect(CollectorRegistry.defaultRegistry)

val program = for {
    r <- externalRegistry
    s <- writeT004(r)
} yield s

val rt = Runtime.unsafeFromLayer(Console.live)

rt.unsafeRun(program >>= (s => putStrLn(s)))

# HELP PrometheusCounter Sample prometheus counter
# TYPE PrometheusCounter counter
# HELP PrometheusHistogram Sample prometheus histogram
# TYPE PrometheusHistogram histogram



import zio.console.putStrLn

import zio.console.Console

import zio.Runtime

import zio.Task


writeT004: CollectorRegistry => Task[String] = ammonite.$sess.cmd17$Helper$$Lambda$3306/361739292@2b818cb4
externalRegistry: Task[CollectorRegistry] = zio.ZIO$EffectPartial@1f139bac
program: zio.ZIO[Any, Throwable, String] = zio.ZIO$FlatMap@73ab4d84
rt: Runtime.Managed[Console] = zio.Runtime$Managed$$anon$1@556eb3e6

In [19]:
import zio.{Queue, UIO, URIO}

val queue: UIO[Queue[String]] = Queue.bounded[String](10)

def send(msg: String)(q: Queue[String]): URIO[Console, Unit] =
  for {
    _ <- putStrLn(s"Sending... $msg")
    _ <- q.offer(msg)
  } yield ()

def sendAsync(msg: String)(q: Queue[String]): UIO[Unit] =
  for {
    _ <- q.offer(msg).fork
  } yield ()

def listen(q: Queue[String]): URIO[Console, Unit] =
  for {
      s <- q.poll
      _ <- putStrLn(s"Received $s")
  } yield ()

import zio.{Queue, UIO, URIO}


queue: UIO[Queue[String]] = zio.ZIO$FlatMap@5024ce2c
defined function send
defined function sendAsync
defined function listen

In [ ]:
val prog0 = for {
    q <- queue
    _ <- URIO.foreach(1 to 20)(i => send(s"Number $i")(q))
} yield ()

rt.unsafeRun(prog0)

In [ ]:
val prog1 = for {
    q <- queue
    _ <- URIO.foreach(1 to 20)(i => sendAsync(s"Number $i")(q))
    _ <- listen(q)
} yield ()

rt.unsafeRun(prog1)

In [20]:
import zio.Schedule

val untilNothing = Schedule.doUntil[String](_ == "Nothing")

def listen(q: Queue[String]): URIO[Console, String] = 
  for {
      s <- q.poll
      _ <- putStrLn(s"Received $s")
  } yield s.getOrElse("Nothing")

val prog2 = for {
    q <- queue
    _ <- URIO.foreach(1 to 20)(i => sendAsync(s"Number $i")(q))
    l <- listen(q).repeat(untilNothing)
} yield l

rt.unsafeRun(prog2 >>= (s => putStrLn(s)))

Received Some(Number 1)
Received Some(Number 2)
Received Some(Number 3)
Received Some(Number 4)
Received Some(Number 5)
Received Some(Number 6)
Received Some(Number 7)
Received Some(Number 8)
Received Some(Number 9)
Received Some(Number 10)
Received Some(Number 11)
Received Some(Number 12)
Received Some(Number 13)
Received Some(Number 14)
Received Some(Number 16)
Received Some(Number 17)
Received Some(Number 18)
Received Some(Number 19)
Received Some(Number 15)
Received Some(Number 20)
Received None
Nothing


import zio.Schedule


untilNothing: Schedule[Any, String, String] = zio.Schedule$$anon$18@5213d165
defined function listen
prog2: zio.ZIO[Console, Nothing, String] = zio.ZIO$FlatMap@3ff46a8f

In [21]:
import zio.RIO

val untilNCollected = Schedule.doUntil[List[String]](_.size == 5)

def listen(q: Queue[String]): RIO[Console, List[String]] = {
  import scala.collection.mutable.ListBuffer
  var collector = new ListBuffer[String]()
  for {
      s <- q.poll
      _ <- putStrLn(s"Received $s")
      if (s.isDefined)
  } yield (collector += s.get).toList // safe cuz we filter on isDefined
}

val prog3 = for {
    q <- queue
    _ <- URIO.foreach(1 to 20)(i => sendAsync(s"Number $i")(q))
    l <- listen(q).repeat(untilNCollected)
} yield l

rt.unsafeRun(prog3 >>= (l => RIO.foreach(l)(s => putStrLn(s"Result: $s"))))

Received Some(Number 1)
Received Some(Number 2)
Received Some(Number 3)
Received Some(Number 4)
Received Some(Number 5)
Result: Number 1
Result: Number 2
Result: Number 3
Result: Number 4
Result: Number 5


import zio.RIO


untilNCollected: Schedule[Any, List[String], List[String]] = zio.Schedule$$anon$18@eae0eca
defined function listen
prog3: zio.ZIO[Console, Throwable, List[String]] = zio.ZIO$FlatMap@2e436a11
res20_4: List[Unit] = List((), (), (), (), ())

In [22]:
import zio.duration._

val everyNSec = Schedule.spaced(2.seconds)
val prog4 = for {
    q <- queue
    _ <- URIO.foreach(1 to 20)(i => sendAsync(s"Number $i")(q))
    l <- listen(q).repeat(everyNSec)
} yield l

rt.unsafeRun(prog4.provideSomeLayer[Console](zio.clock.Clock.live))

Received Some(Number 1)
Received Some(Number 2)
Received Some(Number 3)
Received Some(Number 4)


: 

In [23]:
import zio.clock.Clock
import zio.duration._

val everyNSec = Schedule.spaced(2.seconds)

val rt = Runtime.unsafeFromLayer(Console.live ++ Clock.live)

val everyNSecUntilCollected = untilNCollected && everyNSec

val prog5 = for {
    q <- queue
    _ <- URIO.foreach(1 to 20)(i => sendAsync(s"Number $i")(q))
    l <- listen(q).repeat(everyNSecUntilCollected)
} yield l

rt.unsafeRun(prog5)

Received Some(Number 1)
Received Some(Number 2)
Received Some(Number 3)
Received Some(Number 4)
Received Some(Number 5)


import zio.clock.Clock

import zio.duration._


everyNSec: Schedule[Clock, Any, Int] = zio.Schedule$$anon$18@2dfd9bdd
rt: Runtime.Managed[Console with Clock] = zio.Runtime$Managed$$anon$1@773b0db9
everyNSecUntilCollected: Schedule[Clock, List[String], (List[String], Int)] = zio.Schedule$$anon$1@6f2b5697
prog5: zio.ZIO[Console with Clock, Throwable, (List[String], Int)] = zio.ZIO$FlatMap@7168e0aa
res22_6: (List[String], Int) = (
  List("Number 1", "Number 2", "Number 3", "Number 4", "Number 5"),
  4
)

In [36]:
def listen(q: Queue[String]): RIO[Console, List[String]] = {
  import scala.collection.mutable.ListBuffer
  var collector = new ListBuffer[String]()
  for {
      s <- q.poll
      _ <- putStrLn(s"Received $s")
      if (s.isDefined)
  } yield {
      if (collector.size == 5) collector.clear()
      (collector += s.get).toList
  }
}

val prog6 = for {
    q <- queue
    _ <- URIO.foreach(1 to 20)(i => sendAsync(s"Number $i")(q))
    l <- listen(q).repeat(untilNCollected) <& listen(q).repeat(everyNSec)
} yield l

rt.unsafeRun(prog6)

Collector size = 0
Received Some(Number 1)
Received Some(Number 2)
Received Some(Number 3)
Received Some(Number 4)
Received Some(Number 6)
Received Some(Number 5)
Collector current size: 5
Collector size = 0
Collector size = 0
Received Some(Number 7)
Collector current size: 5
Collector size = 0
Collector size = 0
Received Some(Number 8)
Collector current size: 5
Collector size = 0
Collector size = 0
Received Some(Number 9)
Collector current size: 5
Collector size = 0
Collector size = 0
Received Some(Number 10)
Collector current size: 5
Collector size = 0
Collector size = 0
Received Some(Number 11)
Collector current size: 5
Collector size = 0
Collector size = 0
Received Some(Number 12)
Collector current size: 5
Collector size = 0
Collector size = 0
Received Some(Number 13)
Collector current size: 5
Collector size = 0
Collector size = 0
Received Some(Number 14)
Collector current size: 5
Collector size = 0
Collector size = 0
Received Some(Number 15)
Collector current size: 5
Collector siz

: 

In [37]:
import zio.{ Has, Layer, ZLayer }
import zio.{ RIO, Task }

type Metrics = Has[Metrics.Service]

object Metrics {
    trait Service {
      def getRegistry(): Task[CollectorRegistry]

      def inc(tags: Array[String]): Task[Unit]

      def inc(amount: Double, tags: Array[String]): Task[Unit]

      def time(f: () => Unit, tags: Array[String]): Task[Double]

    }

    val live: Layer[Nothing, Metrics] = ZLayer.succeed(new Service {

      private val (myCounter, myHistogram) = {
          val c = Counter
            .build()
            .name("LayerCounter")
            .labelNames(Array("class", "method"): _*)
            .help(s"Prometheus counter inside a layer")
            .register()

        val h = Histogram
            .build()
            .name("LayerHistogram")
            .labelNames(Array("class", "method"): _*)
            .linearBuckets(0.0, 0.2, 5)
            .help(s"Prometheus histogram inside a layer")
            .register()
          
          (c, h)
      }

      def getRegistry(): Task[CollectorRegistry] =
        Task.effect(CollectorRegistry.defaultRegistry)

      def inc(tags: Array[String]): zio.Task[Unit] =
        inc(1.0, tags)

      def inc(amount: Double, tags: Array[String]): Task[Unit] =
        Task(myCounter.labels(tags:_*)inc(amount))

      def time(f: () => Unit, tags: Array[String]): Task[Double] = Task {
          val t = myHistogram.labels(tags: _*).startTimer()
          f()
          t.observeDuration()
      }
    })
}

import zio.{ Has, Layer, ZLayer }

import zio.{ RIO, Task }


defined type Metrics
defined object Metrics

In [38]:
val program1 = for {
      m  <- RIO.environment[Metrics]
      _  <- putStrLn("Metrics")
      r  <- m.get.getRegistry()
      _  <- m.get.inc(Array("RequestCounter", "get"))
      _  <- m.get.inc(Array("RequestCounter", "post"))
      _  <- m.get.inc(2.0, Array("LoginCounter", "login"))
      _  <- m.get.time(() => Thread.sleep(2000), Array("histogram", "get"))
      s  <- writeT004(r)
      _  <- putStrLn(s)
    } yield ()

program1: zio.ZIO[Metrics with Console, Throwable, Unit] = zio.ZIO$FlatMap@7f4751a5

In [ ]:
CollectorRegistry.defaultRegistry.clear()
rt.unsafeRun(program1.provideSomeLayer[Console](Metrics.live))

In [ ]:
type Metrics2 = Has[Metrics2.Service]

object Metrics2 {
    trait Service {
      def getRegistry(): Task[CollectorRegistry]

      def inc(tags: Array[String]): Task[Unit]

      def inc(amount: Double, tags: Array[String]): Task[Unit]

      def time(f: () => Unit, tags: Array[String]): Task[Double]

    }

    val receiver: (Counter, Histogram) => Layer[Nothing, Metrics2] = 
    (myCounter, myHistogram) => ZLayer.succeed(new Service {

      def getRegistry(): Task[CollectorRegistry] =
        Task.effect(CollectorRegistry.defaultRegistry)

      def inc(tags: Array[String]): zio.Task[Unit] =
        inc(1.0, tags)

      def inc(amount: Double, tags: Array[String]): Task[Unit] =
        Task(myCounter.labels(tags:_*)inc(amount))

      def time(f: () => Unit, tags: Array[String]): Task[Double] = Task {
          val t = myHistogram.labels(tags: _*).startTimer()
          f()
          t.observeDuration()
      }
    })
    
    val receiverHas: ZLayer[Has[(Counter, Histogram)], Nothing, Metrics2] =
      ZLayer.fromFunction[Has[(Counter, Histogram)], Metrics2.Service](
        tup => new Service {
            def getRegistry(): Task[CollectorRegistry] =
              Task.effect(CollectorRegistry.defaultRegistry)

            def inc(tags: Array[String]): zio.Task[Unit] =
              inc(1.0, tags)

            def inc(amount: Double, tags: Array[String]): Task[Unit] =
              Task(tup.get._1.labels(tags:_*)inc(amount))

            def time(f: () => Unit, tags: Array[String]): Task[Double] = Task {
              val t = tup.get._2.labels(tags: _*).startTimer()
              f()
              t.observeDuration()
            }
          }
      )
}

In [ ]:
CollectorRegistry.defaultRegistry.clear()

In [ ]:
val c = Counter
    .build()
    .name("PrometheusCounter")
    .labelNames(Array("class", "method"): _*)
    .help(s"Sample prometheus counter")
    .register()

val h = Histogram
    .build()
    .name("PrometheusHistogram")
    .labelNames(Array("class", "method"): _*)
    .linearBuckets(0.0, 0.2, 5)
    .help(s"Sample prometheus histogram")
    .register()

In [ ]:
val rLayerHas     = ZLayer.succeed[(Counter, Histogram)]((c, h)) >>> Metrics2.receiverHas
val rtReceiverhas = Runtime.unsafeFromLayer(rLayerHas ++ Console.live)

In [ ]:
val rLayer     = Metrics2.receiver(c, h)
val rtReceiver = Runtime.unsafeFromLayer(rLayer ++ Console.live)

In [ ]:
val program2 = for {
      m  <- RIO.environment[Metrics2]
      _  <- putStrLn("Metrics2")
      _  <- m.get.inc(Array("RequestCounter", "get"))
      _  <- m.get.inc(Array("RequestCounter", "post"))
      _  <- m.get.inc(2.0, Array("LoginCounter", "login"))
      _  <- m.get.time(() => Thread.sleep(2000), Array("histogram", "get"))
      r  <- m.get.getRegistry()
      s  <- writeT004(r)
      _  <- putStrLn(s)
    } yield ()

rtReceiverhas.unsafeRun(program2)
//rtReceiver.unsafeRun(program2)